#   Make sure to download the data_test_wo_label.json file from 
#      the CSE 142 website into the ./data folder in order for this 
#      code to run properly

#   Also, if you pulled this version you should have a pickled_feature_vectorizer
#      in your ./src folder

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk as nltk
import numpy as np 
import time;
import csv

In [2]:
result = pd.read_json('../data/data_test_wo_label.json')

# Collecting all the text reviews into a list called corpus. For now, our features will be from the text using sklearn's 
# packages, could add the useful, funny attributes later to see if improves
corpus = []
for index, row in result.iterrows():
    corpus.append(row['text'])
print(len(corpus)) # quick check

50000


In [4]:
ps = PorterStemmer()
test_stemmed = []
for text in corpus:
    words = word_tokenize(text)
    new_text = ""
    for word in words:
        stemmed = ps.stem(word)
        new_text = new_text + " " + stemmed
    test_stemmed.append(new_text)

KeyboardInterrupt: 

In [3]:
import csv
#   A quick insert so the stemming can be done and ready to go without
#   rerunning the previous cell
row_list = []
for each in test_stemmed: #each row consists of one column which is the stemmed text
    text = []
    text.append(each)
    row_list.append(text)
print(len(row_list)) # quick check, needs to be 50000
with open('../data/data_test_wo_labels_stemmed.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

NameError: name 'test_stemmed' is not defined

In [2]:
#   A quick insert so the stemming can be done and ready to go without
#   rerunning the previous cell
#   START HERE
test_stemmed = []
with open('../data/data_test_wo_labels_stemmed.csv', encoding='utf-8') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        test_stemmed.append(row[0])

In [3]:
import pickle
#unload pickled vectorizer with prelearned features from training set
fileObject = open("pickled_feature_vectorizer",'rb')  
# load the object from the file into var b
vectorizer = pickle.load(fileObject)  
fileObject.close()
# print(len(vectorizer.get_feature_names()))
feature_vector = vectorizer.transform(test_stemmed)


In [4]:
#   normalize test feature vectors
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(feature_vector)
normalized_feature_vector = sc.transform(feature_vector)

In [5]:
import scipy.sparse
scipy.sparse.save_npz('../data/data_test_wo_labels_feature_vectors_normalized.npz', normalized_feature_vector)

In [ ]:
# Starting here out will be different

In [2]:
import scipy.sparse
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

In [3]:
labels = []

In [4]:
testset = scipy.sparse.load_npz('../data/data_test_wo_labels_feature_vectors_normalized.npz') #just incase transform takes a while
# unload pickled learned models LR, Perceptron, Nearest_Centroid, SVM(maybe)

fileObject = open("pickled_logistic_regression_model",'rb')
log_reg = pickle.load(fileObject)
fileObject.close()

fileObject = open("pickled_perceptron",'rb')
perceptron = pickle.load(fileObject)
fileObject.close()

fileObject = open("pickled_nearest_centroid",'rb')
nc = pickle.load(fileObject)
fileObject.close()

predictions_LR = log_reg.predict(testset)
predictions_p = perceptron.predict(testset)
predictions_nc = nc.predict(testset)

#   Some stats on the predictions
from collections import Counter
print(Counter(predictions_LR))
print(Counter(predictions_p))
print(Counter(predictions_nc))

labels.append(predictions_LR)
labels.append(predictions_p)
labels.append(predictions_nc)


Counter({'5': 18903, '4': 11986, '1': 7316, '3': 6365, '2': 5430})
Counter({'5': 21623, '1': 8800, '4': 8675, '3': 6348, '2': 4554})
Counter({'5': 17393, '4': 12499, '1': 8022, '3': 6572, '2': 5514})


In [5]:
print(type(labels[0]))

<class 'numpy.ndarray'>


In [6]:
print(labels[0].size)

50000


In [7]:
# voting/ensemble method
final_predictions = [] # then, write to a csv for final predictions
for instance in range(labels[0].size):
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0
    count5 = 0
    for model in range(3):

        vote = labels[model].item(instance)
        if(vote == '1'):
            count1 = count1+1
        if(vote == '2'):
            count2 = count2+1
        if(vote == '3'):
            count3 = count3+1
        if(vote == '4'):
            count4 = count4+1
        if(vote == '5'):
            count5 = count5+1


    votes = [count1, count2, count3, count4, count5]
    winner_index = 0
    for v in range(len(votes)):
        if(votes[v] > votes[winner_index]):
            winner_index = v
    final_predictions.append(winner_index+1)
             
print(Counter(final_predictions))
final_predictions

Counter({5: 18724, 4: 10405, 1: 9193, 3: 6244, 2: 5434})


[4,
 5,
 4,
 4,
 4,
 1,
 5,
 5,
 5,
 5,
 2,
 5,
 5,
 5,
 5,
 3,
 5,
 5,
 5,
 3,
 1,
 3,
 1,
 5,
 5,
 5,
 5,
 1,
 5,
 5,
 2,
 4,
 5,
 1,
 3,
 1,
 5,
 5,
 2,
 2,
 5,
 3,
 5,
 5,
 5,
 5,
 5,
 4,
 5,
 2,
 5,
 3,
 1,
 1,
 1,
 1,
 1,
 5,
 5,
 5,
 2,
 3,
 4,
 5,
 4,
 1,
 1,
 4,
 2,
 5,
 4,
 4,
 1,
 3,
 5,
 3,
 5,
 2,
 4,
 5,
 5,
 1,
 5,
 1,
 4,
 5,
 1,
 4,
 4,
 2,
 1,
 5,
 4,
 5,
 4,
 5,
 5,
 4,
 5,
 4,
 2,
 5,
 3,
 5,
 4,
 5,
 3,
 2,
 3,
 5,
 4,
 5,
 2,
 5,
 4,
 4,
 5,
 1,
 2,
 5,
 1,
 1,
 2,
 5,
 4,
 1,
 5,
 4,
 2,
 4,
 1,
 1,
 4,
 4,
 5,
 1,
 4,
 5,
 4,
 1,
 5,
 2,
 5,
 5,
 5,
 5,
 1,
 4,
 5,
 4,
 4,
 5,
 5,
 2,
 5,
 2,
 2,
 5,
 5,
 5,
 4,
 5,
 5,
 5,
 4,
 4,
 5,
 5,
 4,
 1,
 5,
 4,
 1,
 3,
 5,
 3,
 1,
 5,
 2,
 3,
 5,
 5,
 1,
 3,
 5,
 2,
 4,
 4,
 5,
 5,
 5,
 4,
 5,
 3,
 4,
 3,
 3,
 5,
 5,
 5,
 1,
 2,
 2,
 4,
 3,
 4,
 5,
 5,
 5,
 4,
 5,
 1,
 5,
 5,
 5,
 5,
 2,
 2,
 1,
 1,
 4,
 4,
 5,
 4,
 5,
 4,
 4,
 1,
 5,
 5,
 5,
 2,
 4,
 1,
 1,
 1,
 5,
 1,
 5,
 5,
 1,
 1,
 4,
 5,
 2,
 5,
 1,
 5,
 1,
 5,


In [ ]:
from sklearn.ensemble import VotingClassifier

models=[('lr', log_reg), ('p', perceptron), ('nc', nc)]
ensemble = VotingClassifier(models, voting='soft') #try hard later